<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled109.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

# --- 1. Environment Class ---
class Environment:
    def __init__(self, total_sim_steps, days_per_cycle=50):
        self.total_sim_steps, self.days_per_cycle, self.time = total_sim_steps, days_per_cycle, 0
        num_years = max(1, total_sim_steps / (365 * days_per_cycle))
        steps_per_year = 365 * days_per_cycle
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        full_uv, full_resources = [], []
        for _ in range(int(np.ceil(num_years))):
            uv_severity = np.random.uniform(0.8, 1.25)
            resource_bounty = np.random.uniform(0.8, 1.25)
            seasonal_cycle = np.sin(np.linspace(0, 2 * np.pi, steps_per_year))
            seasonal_amplitude_mod = (0.225 * seasonal_cycle + 0.725) * uv_severity
            seasonal_resource_mod = (-0.20 * seasonal_cycle + 0.80) * resource_bounty
            year_resources = 1500 * seasonal_resource_mod
            year_uv = [daily_uv_cycle[i % days_per_cycle] * seasonal_amplitude_mod[i] for i in range(steps_per_year)]
            full_uv.extend(year_uv); full_resources.extend(year_resources)
        self.uv_intensity = np.array(full_uv)[:total_sim_steps]
        self.external_resources = np.array(full_resources)[:total_sim_steps]

    def get_current_uv(self):
        return self.uv_intensity[self.time] if self.time < self.total_sim_steps else 0.0
    def get_current_resources(self):
        return self.external_resources[self.time] if self.time < self.total_sim_steps else 0.0
    def step(self):
        self.time += 1

# --- 2. Protoribosome Class ---
class Protoribosome:
    def __init__(self, env, strategy, initial_sequence, initial_mass=100.0):
        self.env, self.strategy, self.rna_sequence = env, strategy, list(initial_sequence)
        self.rna_mass = initial_mass
        self.atp_pool, self.mcyw_peptide_pool, self.other_peptide_pool = 500.0, 50.0, 5.0
        self.internal_ntp_stock, self.internal_aa_stock, self.internal_trp_tyr_cys_stock = 200.0, 200.0, 100.0
        self.rna_damage_level = 0.0

        self.location = 'dark_zone'
        self.status = 'ACTIVE'

        self.metabolic_efficiency = 1.0
        self.uv_damage_rate_per_uv = 0.5
        self.repair_rate, self.repair_atp_cost, self.trp_tyr_cys_for_repair_needed = 0.5, 0.0025, 0.0005
        self.uv_protection_factor, self.damage_tolerance_threshold = 0.01, 3.0
        self.atp_generation_rate_per_mcyw_uv, self.baseline_atp_generation_rate = 0.15, 0.5
        self.baseline_atp_chemosynthesis = 1.5
        self.protein_synthesis_rate, self.protein_synthesis_ntp_cost, self.protein_synthesis_aa_cost, self.protein_synthesis_atp_cost = 0.20, 0.1, 0.1, 0.02
        self.replication_rate, self.replication_ntp_cost, self.replication_atp_cost = 0.05, 1.0, 0.2
        self.rna_degradation_rate, self.atp_degradation_rate, self.peptide_degradation_rate = 0.0005, 0.025, 0.0008
        self.diffusion_rate, self.atp_from_ntp_conversion = 0.30, 0.1
        self.mutation_prob = 0.0008
        self.failed_synthesis_penalty_atp = 5.0
        # --- NEW: Parameters for individual growth ---
        self.growth_rate = 0.1
        self.growth_ntp_cost = 1.0
        self.growth_atp_cost = 0.2

    def get_peptide_type(self):
        codons = ["".join(self.rna_sequence[i:i+3]) for i in range(0, len(self.rna_sequence), 3)]
        if any(c in {'UGU', 'UGC', 'UAU', 'UAC', 'UGG'} for c in codons):
            return "MCYW"
        return "XXXX"

    def has_stop_codon(self):
        return any(c in {'UAA', 'UAG', 'UGA'} for c in ["".join(self.rna_sequence[i:i+3]) for i in range(0, len(self.rna_sequence), 3)])

    def _decide_zone_transition(self):
        if self.location == 'dark_zone' and self.atp_pool < 250:
            self.location = 'light_zone'
        elif self.location == 'light_zone' and self.rna_damage_level > self.damage_tolerance_threshold * 0.75:
            self.location = 'dark_zone'

    def _apply_uv_damage(self, current_uv):
        if self.location == 'dark_zone' or current_uv <= 0:
            self.rna_damage_level = max(0, self.rna_damage_level - self.repair_rate * 0.1)
            return

        protection = self.mcyw_peptide_pool * self.uv_protection_factor
        effective_uv = current_uv * max(0.01, 1 - protection)
        self.rna_damage_level += effective_uv * self.uv_damage_rate_per_uv

        for i in range(len(self.rna_sequence)):
            if random.random() < self.mutation_prob * effective_uv:
                self.rna_sequence[i] = random.choice("AUGC")

    def _manage_state(self):
        if self.atp_pool < 0.1 or self.rna_damage_level >= self.damage_tolerance_threshold:
            self.status = 'INACTIVE'
            return

        stop_signal = self.has_stop_codon()
        if self.strategy == 'cautious' and stop_signal:
            self.status = 'ARRESTED'
            self.metabolic_efficiency = 1.0
        else:
            self.status = 'ACTIVE'
            self.metabolic_efficiency = 1.25

    def _synthesize_protein(self):
        if self.status != 'ACTIVE': return
        units = self.protein_synthesis_rate * self.metabolic_efficiency * 10
        required_ntp = units * self.protein_synthesis_ntp_cost
        required_atp = units * self.protein_synthesis_atp_cost
        required_aa_total = units * self.protein_synthesis_aa_cost

        is_mcyw = self.get_peptide_type() == "MCYW" and not (self.strategy == 'readthrough' and self.has_stop_codon())

        if is_mcyw:
            required_special_aa = required_aa_total * 0.5
            required_general_aa = required_aa_total - required_special_aa
            if (self.internal_ntp_stock < required_ntp or
                self.atp_pool < required_atp or
                self.internal_aa_stock < required_general_aa or
                self.internal_trp_tyr_cys_stock < required_special_aa):
                return
        else:
            required_general_aa = required_aa_total
            required_special_aa = 0
            if (self.internal_ntp_stock < required_ntp or
                self.atp_pool < required_atp or
                self.internal_aa_stock < required_general_aa):
                return

        self.internal_ntp_stock -= required_ntp
        self.atp_pool -= required_atp
        self.internal_aa_stock -= required_general_aa
        self.internal_trp_tyr_cys_stock -= required_special_aa

        if is_mcyw:
            self.mcyw_peptide_pool += units
        else:
            self.other_peptide_pool += units
            if self.strategy == 'readthrough' and self.has_stop_codon():
                self.atp_pool -= self.failed_synthesis_penalty_atp

    def _replicate_rna(self, dna_template, template_health):
        # Only replicate if mass is high enough
        if self.status != 'ACTIVE' or self.rna_mass < 80: return None

        effective_replication_rate = self.replication_rate * template_health
        can_replicate = self.rna_mass * effective_replication_rate * self.metabolic_efficiency

        if self.internal_ntp_stock > self.replication_ntp_cost and self.atp_pool > self.replication_atp_cost and can_replicate > 1.0:
            # Shed a portion of mass for offspring
            offspring_mass = self.rna_mass * 0.5
            self.rna_mass -= offspring_mass

            # Costs are proportional to the new mass created
            self.internal_ntp_stock -= offspring_mass * self.replication_ntp_cost
            self.atp_pool -= offspring_mass * self.replication_atp_cost
            return Protoribosome(self.env, self.strategy, "".join(self.rna_sequence), offspring_mass)
        return None

    def _repair_rna_dna(self, dna_template):
        if self.status != 'ACTIVE' or self.get_peptide_type() != 'MCYW' or self.has_stop_codon():
            return False

        if "".join(self.rna_sequence) != dna_template:
            if self.atp_pool >= self.repair_atp_cost and self.internal_trp_tyr_cys_stock >= self.trp_tyr_cys_for_repair_needed:
                self.atp_pool -= self.repair_atp_cost
                self.internal_trp_tyr_cys_stock -= self.trp_tyr_cys_for_repair_needed
                self.rna_sequence = list(dna_template)
                self.rna_damage_level = 0
                return True
        return False

    def _diffuse_resources(self):
        available = self.env.get_current_resources()
        if available > 0:
            amount = available * self.diffusion_rate
            self.internal_ntp_stock += amount * 0.50
            self.internal_aa_stock += amount * 0.45
            self.internal_trp_tyr_cys_stock += amount * 0.05

    # --- NEW METHOD: Allows individual protoribosomes to grow ---
    def _grow_mass(self):
        if self.status != 'ACTIVE': return

        potential_growth = self.rna_mass * self.growth_rate * self.metabolic_efficiency
        ntp_cost = potential_growth * self.growth_ntp_cost
        atp_cost = potential_growth * self.growth_atp_cost

        if self.internal_ntp_stock >= ntp_cost and self.atp_pool >= atp_cost:
            self.internal_ntp_stock -= ntp_cost
            self.atp_pool -= atp_cost
            self.rna_mass += potential_growth

    def step(self, current_uv, dna_template, template_health):
        if self.status == 'INACTIVE': return False, None

        self._decide_zone_transition()
        self._apply_uv_damage(current_uv)
        self._manage_state()
        if self.status == 'INACTIVE': return False, None

        self._diffuse_resources()
        self._grow_mass() # Grow mass before other actions

        self.atp_pool += self.baseline_atp_chemosynthesis
        self.atp_pool *= (1 - self.atp_degradation_rate)

        if self.location == 'light_zone' and current_uv > 0:
            self.atp_pool += (self.atp_generation_rate_per_mcyw_uv * self.mcyw_peptide_pool * current_uv)

        self._synthesize_protein()
        was_repaired = self._repair_rna_dna(dna_template)
        new_offspring = self._replicate_rna(dna_template, template_health)

        return was_repaired, new_offspring

# --- 3. Colony Class ---
class Colony:
    def __init__(self, env, dna_template, initial_pop_size=50):
        self.env = env
        self.dna_template = dna_template
        self.population = []
        for i in range(initial_pop_size):
            strategy = 'cautious' if i % 2 == 0 else 'readthrough'
            self.population.append(Protoribosome(env, strategy, dna_template))

        self.template_health = 1.0
        self.template_repair_rate = 0.05
        self.template_decay_rate = 0.001

    def step(self):
        newly_replicated = []
        successful_repairs = 0

        current_uv = self.env.get_current_uv()

        for protoribosome in list(self.population):
            was_repaired, new_offspring = protoribosome.step(current_uv, self.dna_template, self.template_health)

            if was_repaired:
                successful_repairs += 1
            if new_offspring:
                newly_replicated.append(new_offspring)

        self.template_health += successful_repairs * self.template_repair_rate
        self.template_health *= (1 - self.template_decay_rate)
        self.template_health = min(1.0, self.template_health)

        self.population.extend(newly_replicated)

    def get_aggregated_data(self):
        data = { 'cautious': 0, 'readthrough': 0, 'inactive': 0, 'in_light_zone': 0 }
        if not self.population: return data
        for p in self.population:
            if p.status == 'INACTIVE':
                data['inactive'] += 1
            elif p.strategy == 'cautious':
                data['cautious'] += 1
            elif p.strategy == 'readthrough':
                data['readthrough'] += 1
            if p.location == 'light_zone':
                data['in_light_zone'] += 1
        return data

# --- 4. Simulation and Plotting ---
def run_and_plot():
    SIMULATION_DAYS = 20
    STEPS_PER_DAY = 20
    TOTAL_STEPS = SIMULATION_DAYS * STEPS_PER_DAY

    IDEAL_DNA_TEMPLATE = "AUGUGUUACUGG"

    env = Environment(TOTAL_STEPS, STEPS_PER_DAY)
    colony = Colony(env, IDEAL_DNA_TEMPLATE, initial_pop_size=150)

    data = []
    print_interval = max(1, TOTAL_STEPS / 1000)

    for i in range(TOTAL_STEPS):
        if i % int(print_interval) == 0:
            agg_data = colony.get_aggregated_data()
            cautious_pop = agg_data.get('cautious', 0)
            readthrough_pop = agg_data.get('readthrough', 0)
            inactive_pop = agg_data.get('inactive', 0)

            print(f"Simulating step {i}/{TOTAL_STEPS} ({(i/TOTAL_STEPS)*100:.1f}%)... "
                  f"Cautious: {cautious_pop} | "
                  f"Readthrough: {readthrough_pop} | "
                  f"Inactive: {inactive_pop}")

            if i > 200 and (cautious_pop + readthrough_pop == 0):
                print("Active population extinct. Ending simulation early.")
                break

        colony.step()

        log_interval = 1 if TOTAL_STEPS <= 4000 else STEPS_PER_DAY
        if i % log_interval == 0:
            agg_data = colony.get_aggregated_data()
            log_entry = {
                'time': env.time,
                'uv': env.get_current_uv(),
                'template_health': colony.template_health
            }
            log_entry.update(agg_data)
            data.append(log_entry)

        env.step()

    if not data:
        print("No data was logged. Cannot generate plots.")
        return

    results_df = pd.DataFrame(data)

    fig, axs = plt.subplots(3, 1, figsize=(15, 12), sharex=True)
    fig.suptitle('Protoribosome Colony Simulation (20 Days)', fontsize=16)
    colors = {'cautious': 'orange', 'readthrough': 'purple', 'inactive': 'grey'}

    ax0_twin = axs[0].twinx()
    axs[0].plot(results_df['time'], results_df['uv'], color='black', alpha=0.2, label='UV Intensity')
    ax0_twin.plot(results_df['time'], results_df['template_health'], color='cyan', label='Template Health')
    axs[0].set_title('Environmental Conditions & DNA Health'); axs[0].set_ylabel('UV'); ax0_twin.set_ylabel('Health')
    axs[0].legend(loc='upper left'); ax0_twin.legend(loc='upper right')

    axs[1].plot(results_df['time'], results_df['cautious'], label='Cautious', color=colors['cautious'])
    axs[1].plot(results_df['time'], results_df['readthrough'], label='Readthrough', color=colors['readthrough'])
    axs[1].set_title('Active Population by Strategy'); axs[1].set_ylabel('Count'); axs[1].legend()
    axs[1].set_yscale('symlog')

    axs[2].stackplot(results_df['time'],
                     results_df['cautious'] + results_df['readthrough'],
                     results_df['inactive'],
                     labels=['Active', 'Inactive'],
                     colors=[colors['cautious'], colors['inactive']])
    axs[2].set_title('Total Population State (Active vs. Inactive)'); axs[2].set_ylabel('Total Count'); axs[2].legend(loc='upper left')

    for ax in axs:
        ax.grid(True, linestyle=':', linewidth='0.5', color='gray')
        ax.set_xlim(0, TOTAL_STEPS)

    plt.tight_layout(rect=[0, 0, 1, 0.96]); plt.show()

if __name__ == "__main__":
    run_and_plot()


Simulating step 0/400 (0.0%)... Cautious: 75 | Readthrough: 75 | Inactive: 0
Simulating step 1/400 (0.2%)... Cautious: 150 | Readthrough: 150 | Inactive: 0
Simulating step 2/400 (0.5%)... Cautious: 150 | Readthrough: 150 | Inactive: 0
Simulating step 3/400 (0.8%)... Cautious: 150 | Readthrough: 150 | Inactive: 0
Simulating step 4/400 (1.0%)... Cautious: 300 | Readthrough: 300 | Inactive: 0
Simulating step 5/400 (1.2%)... Cautious: 300 | Readthrough: 300 | Inactive: 0
Simulating step 6/400 (1.5%)... Cautious: 300 | Readthrough: 300 | Inactive: 0
Simulating step 7/400 (1.8%)... Cautious: 300 | Readthrough: 300 | Inactive: 0
Simulating step 8/400 (2.0%)... Cautious: 300 | Readthrough: 300 | Inactive: 0
Simulating step 9/400 (2.2%)... Cautious: 300 | Readthrough: 300 | Inactive: 0
Simulating step 10/400 (2.5%)... Cautious: 600 | Readthrough: 600 | Inactive: 0
Simulating step 11/400 (2.8%)... Cautious: 600 | Readthrough: 600 | Inactive: 0
Simulating step 12/400 (3.0%)... Cautious: 600 | Rea